In [1]:
using LinearAlgebra
using DynamicPolynomials

dim = 3
M = [0 0 1.0; 1.0 0 0; 0 1 0]

@polyvar x[1:dim] # nonnegative variables
@polyvar y[1:dim]
@polyvar d[1:dim]

z=[x;y;d]

f=sum(1.0*d) # the objective polynomial to minimize

g=M*y-x # the inequality constraints
h=[1.0*y-d.*x;sum(x)-1.0;d[1]-1;d[2]-1;d[3]-1] # the equality constraints

k=1 # relaxation order
s=3 # sparsity order


include("../src/InterRelax.jl")
using .InterRelax



# get information from the input data f,gi,hj
n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(z,f,g,h,sparse=false);

# get an approximate optimal value and an approximate optimal solution of the polynomial optimization problem
opt_val,opt_sol=InterRelax.RelaxDense(n,m,l,
                                      lmon_g,supp_g,coe_g, # information of the inequality constraints
                                      lmon_h,supp_h,coe_h, # information of the equality constraints
                                      lmon_f,supp_f,coe_f, # information of the objective polynomial
                                      dg,dh,k,s,
                                      solver="Mosek", # solver for the semidefinite program
                                      comp_opt_sol=true) # to get an approximate optimal solution

┌ Info: Precompiling DynamicPolynomials [7c1d4256-1411-5781-91ec-d7bc3513ac07]
└ @ Base loading.jl:1273
┌ Info: Precompiling MosekTools [1ec41992-ff65-5c91-ac43-2df89e9693a4]
└ @ Base loading.jl:1273
┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1273
┌ Info: Precompiling SumOfSquares [4b9e565b-77fc-50a5-a571-1244f986bda1]
└ @ Base loading.jl:1273
┌ Info: Precompiling LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d]
└ @ Base loading.jl:1273
┌ Info: Precompiling COSMO [1e616198-aa4e-51ec-90a2-23f7fbd31d8d]
└ @ Base loading.jl:1273
┌ Info: Precompiling PowerModels [c36e90e8-916a-50a6-bd94-075b64ef4655]
└ @ Base loading.jl:1273
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1273
┌ Info: Precompiling MatrixMarket [4d4711f2-db25-561a-b6b3-d35e7d4047d3]
└ @ Base loading.jl:1273
  likely near /home/hoanganh/.julia/packages/MatrixMarket/ic62N/src/MatrixMarket.jl:3
  likely near /home/hoanganh/.julia/packages/MatrixM

**Interrupted relaxation based on Polya's Positivstellensatz**
Relaxation order: k=1
Factor width upper bound: s=3
Maximal matrix size:3
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 55              
  Cones                  : 0               
  Scalar variables       : 119             
  Matrix variables       : 8               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 24
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 1
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number

(2.999999993934861, Array{Float64,1}[])

In [2]:
@polyvar w[1:dim]

h=[h;f-opt_val;g-w.^2]#polynomial equalities determining the set of solution 

11-element Array{Polynomial{true,Float64},1}:
 -x₁d₁ + y₁                      
 -x₂d₂ + y₂                      
 -x₃d₃ + y₃                      
 x₁ + x₂ + x₃ - 1.0              
 d₁ - 1.0                        
 d₂ - 1.0                        
 d₃ - 1.0                        
 d₁ + d₂ + d₃ - 2.999999993934861
 -w₁² - x₁ + y₃                  
 -w₂² - x₂ + y₁                  
 -w₃² - x₃ + y₂                  

In [3]:
using HomotopyContinuation

@var x_new[1:3] y_new[1:3] d_new[1:3] w_new[1:3]

h_new=[-x_new[1]*d_new[1] + y_new[1],                
 -x_new[2]*d_new[2] + y_new[2],                
 -x_new[3]*d_new[3] + y_new[3],                      
 x_new[1] + x_new[2] + x_new[3] - 1,            
 d_new[1] - 1,          
 d_new[2] - 1,                        
 d_new[3] - 1,                       
 d_new[1] + d_new[2] + d_new[3] - 3,
-w_new[1]^2 - x_new[1] + y_new[3],
-w_new[2]^2 - x_new[2] + y_new[1],
-w_new[3]^2 - x_new[3] + y_new[2]]

result = solve(System(h_new))

┌ Info: Precompiling HomotopyContinuation [f213a82b-91d6-5c5d-acf7-10f1c761b327]
└ @ Base loading.jl:1273


MethodError: MethodError: no method matching System(::Array{HomotopyContinuation.ModelKit.Operation,1})
Closest candidates are:
  System(::Array{#s126,1} where #s126<:HomotopyContinuation.ModelKit.Expression, !Matched::Array{HomotopyContinuation.ModelKit.Variable,1}) at /home/hoanganh/.julia/packages/HomotopyContinuation/mJDHb/src/model_kit/expression.jl:680
  System(::Array{#s125,1} where #s125<:HomotopyContinuation.ModelKit.Expression, !Matched::Array{HomotopyContinuation.ModelKit.Variable,1}, !Matched::Array{HomotopyContinuation.ModelKit.Variable,1}) at /home/hoanganh/.julia/packages/HomotopyContinuation/mJDHb/src/model_kit/expression.jl:680
  System(!Matched::Array{HomotopyContinuation.ModelKit.Expression,1}, !Matched::Array{HomotopyContinuation.ModelKit.Variable,1}, !Matched::Array{HomotopyContinuation.ModelKit.Variable,1}) at /home/hoanganh/.julia/packages/HomotopyContinuation/mJDHb/src/model_kit/expression.jl:669

In [47]:
#=k=2 # relaxation order
s=500 # sparsity order

a=zeros(Float64,9)
# get information from the input data f,gi,hj
n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(z,sum((z-a).^2),g,h,sparse=false);

# get an approximate optimal value and an approximate optimal solution of the polynomial optimization problem
val,opt_sol=InterRelax.RelaxDense(n,m,l,
                                lmon_g,supp_g,coe_g, # information of the inequality constraints
                                  lmon_h,supp_h,coe_h, # information of the equality constraints
                                  lmon_f,supp_f,coe_f, # information of the objective polynomial
                                  dg,dh,k,s,
                                  solver="Mosek", # solver for the semidefinite program
                                  comp_opt_sol=true) # to get an approximate optimal solution
h=[h;sum((a-z).^2)-val]=#

In [48]:
#=a_vec=Vector{Vector{Float64}}(undef,9)
val_vec=Vector{Float64}(undef,9)

for j=1:9
    println("Sphere ", j)
    a_vec[j]=a
    a_vec[j][j]=j
    # get information from the input data f,gi,hj
    n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(z,sum((z-a_vec[j]).^2),g,h,sparse=false);

    # get an approximate optimal value and an approximate optimal solution of the polynomial optimization problem
    val_vec[j],opt_sol=InterRelax.RelaxDense(n,m,l,
                                          lmon_g,supp_g,coe_g, # information of the inequality constraints
                                      lmon_h,supp_h,coe_h, # information of the equality constraints
                                      lmon_f,supp_f,coe_f, # information of the objective polynomial
                                      dg,dh,k,s,
                                      solver="Mosek", # solver for the semidefinite program
                                      comp_opt_sol=true) # to get an approximate optimal solution
    h=[h;sum((a_vec[j]-z).^2)-val_vec[j]]
    println()
    println("--------------")
    println()
end=#